## Import

In [1]:
import random
import torch
import os
import time
import datetime
import shutil

import numpy as np
import pandas as pd
from IPython.display import HTML

from torch.optim import AdamW
from torch.utils.data import Dataset as TorchDataset
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler
)
from transformers import (
    get_linear_schedule_with_warmup,
    T5Tokenizer,
    T5ForConditionalGeneration
)
from sklearn.metrics import (
    f1_score,
    matthews_corrcoef
)

print('Import is done.')

C:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import is done.


## Config

In [2]:
RANDOM_SEED = 42

TRAIN_KEY = 'train'
TEST_KEY = 'test'
VAL_KEY = 'val'

DATA_DIRECTORY = '.\\data'
DANETQA_INPUT = os.path.join(DATA_DIRECTORY, 'danetqa_paths.json')

SAVED_MODEL_PATH = os.path.join(DATA_DIRECTORY, 'saved')

PREPARED_BERT_PATH = 'ai-forever/ruBert-base'
PREPARED_T5_BASE_PATH = 'ai-forever/ruT5-base'

LR = 2e-5
EPS = 1e-8
TRAIN_FRAC = 0.9
VAL_FRAC = 0.9
BATCH_SIZE = 32
EPOCHS = 1

SENTENCE_COLS = ['question', 'passage']

USE_RAW_MAX_LENGTH = True
USE_BY_THRESHOLD = True
TOKEN_SIZE_THRESHOLD = 300

TESTING_FROM_FILES = True

print('Constants are initialized.')

Constants are initialized.


## Set random seed

In [3]:
random.seed(RANDOM_SEED) # check needness
np.random.seed(RANDOM_SEED) # check needness
torch.manual_seed(RANDOM_SEED) # check needness
torch.cuda.manual_seed(RANDOM_SEED) # check needness

print('Random seed is set.')

Random seed is set.


## Define device

In [4]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    device = torch.device('cuda')
else:
    print('No GPU available, using the GPU instead.')
    device = torch.device('cpu')
    
print('Device is defined.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3080 Laptop GPU
Device is defined.


## Model & tokenizer & optimizer creation

In [5]:
tokenizer = T5Tokenizer.from_pretrained(PREPARED_T5_BASE_PATH, use_fast=False)
model = T5ForConditionalGeneration.from_pretrained(PREPARED_T5_BASE_PATH)
optimizer = AdamW(model.parameters(), lr=LR, eps=EPS)


print('Model, tokenizer, optimizer are created.')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Model, tokenizer, optimizer are created.


## Loading & preparation of dataframes

In [6]:
paths_df = pd.read_json(DANETQA_INPUT, lines=True)

train_file_path = paths_df.get(TRAIN_KEY).values[0]
test_file_path = paths_df.get(TEST_KEY).values[0]
val_file_path = paths_df.get(VAL_KEY).values[0]

original_train_df = pd.read_json(train_file_path, lines=True)
original_val_df = pd.read_json(val_file_path, lines=True)
original_test_df = pd.read_json(test_file_path, lines=True)

output_df = pd.DataFrame(
    [
        [TRAIN_KEY, train_file_path, len(original_train_df)],
        [TEST_KEY, test_file_path, len(original_val_df)],
        [VAL_KEY, val_file_path, len(original_test_df)]
    ],
    columns=['Key', 'Path', 'Size']
)
print('Original datasets:')
display(HTML(output_df.to_html(index=False)))

def enrich_df(df) -> None:
    df['sentence'] = df[SENTENCE_COLS].apply(lambda row: ''.join(row).lower(), axis=1)
    df['l'] = df[['label']].apply(lambda label: 1 if label[0] else 0, axis=1)

random_index = original_train_df.sample(frac=TRAIN_FRAC, random_state=RANDOM_SEED).index
test_df0 = original_train_df[~original_train_df.index.isin(random_index)]
train_df = original_train_df[original_train_df.index.isin(random_index)].reset_index(drop=True)

random_index = original_val_df.sample(frac=VAL_FRAC, random_state=RANDOM_SEED).index
test_df1 = original_val_df[~original_val_df.index.isin(random_index)]
val_df = original_val_df[original_val_df.index.isin(random_index)].reset_index(drop=True)

test_df = pd.concat([test_df0, test_df1], ignore_index=True)

enrich_df(train_df)
enrich_df(val_df)
enrich_df(test_df)

output_df = pd.DataFrame(
    [
        [TRAIN_KEY, len(train_df)],
        [TEST_KEY, len(val_df)],
        [VAL_KEY, len(test_df)]
    ],
    columns=['Key', 'Size']
)
print('Used datasets:')
display(HTML(output_df.to_html(index=False)))

print('Loading & preparation of dataframes are done.')

Original datasets:


Key,Path,Size
train,.\data\DaNetQA\train.jsonl,1749
test,.\data\DaNetQA\test.jsonl,821
val,.\data\DaNetQA\val.jsonl,805


Used datasets:


Key,Size
train,1574
test,739
val,257


Loading & preparation of dataframes are done.


## Define max_length

In [7]:
def define_raw_max_length_by_bert(sentences, raw_max_length):
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, add_special_tokens=True)
        raw_max_length = max(raw_max_length, len(input_ids))    
    return raw_max_length

def define_max_length(raw_max_length, threshold):
    return threshold if threshold >= raw_max_length else define_max_length(raw_max_length, threshold * 2)

raw_max_length = define_raw_max_length_by_bert(train_df.sentence.values, 0)
raw_max_length = define_raw_max_length_by_bert(val_df.sentence.values, raw_max_length)
raw_max_length = define_raw_max_length_by_bert(test_df.sentence.values, raw_max_length)

binary_based_max_length = define_max_length(raw_max_length, 1)
if USE_BY_THRESHOLD:
    max_length = TOKEN_SIZE_THRESHOLD
else:
    max_length = raw_max_length if USE_RAW_MAX_LENGTH else binary_based_max_length

output_df = pd.DataFrame(
    [
        ['Raw', raw_max_length],
        ['Binary-based', binary_based_max_length],
        ['By-threshold', TOKEN_SIZE_THRESHOLD],
        ['Used', max_length]
    ],
    columns=['Type', 'Size']
)
print('Definition of max_length:')
display(HTML(output_df.to_html(index=False)))

print('Maximum length of sentences is defined.')

Definition of max_length:


Type,Size
Raw,838
Binary-based,1024
By-threshold,300
Used,300


Maximum length of sentences is defined.


## Datasets & dataloaders creation

In [15]:
class T5TestDataset(TorchDataset):
    def __init__(self, text, tokenizer, length, device):
        self._text = text.reset_index(drop=True)
        self._tokenizer = tokenizer
        self._length = length
        self._device = device

    def __len__(self):
        #<
#         print('len')
        return self._text.shape[0]

    def __getitem__(self, item):
        #< 
#         print('__getitem__')
        output = self._tokenize(self._text[item])
        return {k: v.reshape(-1).to(self._device) for k, v in output.items()}

    def _tokenize(self, text):
        #< 
#         print('_tokenize')
        return self._tokenizer(text,
                               return_tensors='pt',
                               padding='max_length',
                               truncation=True,
                               max_length=self._length)


class T5TrainDataset(TorchDataset):
    POS_LABEL = 'верно'
    NEG_LABEL = 'неверно'

    def __init__(self, text, label, tokenizer, length, device):
        self._text = text.reset_index(drop=True)
        self._label = label.reset_index(drop=True)
        self._tokenizer = tokenizer
        self._length = length
        self._device = device

    def __len__(self):
        return self._label.shape[0]

    def __getitem__(self, item):
        output = self._tokenize(self._text[item], self._length)
        output = {k: v.reshape(-1).to(self._device) for k, v in output.items()}

        label = self.POS_LABEL if self._label[item] == 1 else self.NEG_LABEL
        label = self._tokenize(label, length=2).input_ids.reshape(-1).to(self._device)

        output.update({'labels': label})
        return output

    def _tokenize(self, text, length):
        return self._tokenizer(text,
                               return_tensors='pt',
                               padding='max_length',
                               truncation=True,
                               max_length=length)

    
train_dataset = T5TrainDataset(train_df['sentence'], train_df['l'], tokenizer, max_length, device)
val_dataset = T5TrainDataset(val_df['sentence'], val_df['l'], tokenizer, max_length, device)
test_dataset = T5TestDataset(test_df['sentence'], tokenizer, max_length, device)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print('Datasets & dataloaders creation is done')

Datasets & dataloaders creation is done


## Scheduler creation

In [9]:
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * EPOCHS
)

print('Scheduler is created')

Scheduler is created


## Training

In [21]:
model.to(device)
# n_epochs = conf('train.epochs.t5')

dl_length = len(train_dataloader)

# <
# print(f'dataloader len: {dl_length}')

for epoch in range(EPOCHS):
    print(f'\n======= Epoch {epoch + 1} / {EPOCHS} =======\n')

    print('Training...\n')
    model.train()
    
    for batch_id, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        # < del
#         print(f'{batch_id} {datetime.datetime.now()}')
        if (((batch_id + 1) % 10 == 0) and not batch_id == 0) or (batch_id == dl_length - 1):
            print(f'\t\tBatch {batch_id+1} of {dl_length}, loss : {loss.item():.3f}')
        
    print('Validation...')
    model.eval()
    
    with torch.no_grad():
        eval_loss = [model(**batch).loss.item() for batch in val_dataloader]
        
    print(f'\tValidation loss: {np.sum(eval_loss)/len(eval_loss)}')
    
print('Trainig complete!')


======= Epoch 1 / 1 =======

Training...

		Batch 10 of 50, loss : 5.585
		Batch 20 of 50, loss : 2.458
		Batch 30 of 50, loss : 1.766
		Batch 40 of 50, loss : 0.740
		Batch 50 of 50, loss : 1.253
Validation...
	Validation loss: 0.4562477357685566
Trainig complete!


## Testing

In [16]:
def test(model, tokenizer, dataloader):
    pos_label = tokenizer(T5TrainDataset.POS_LABEL,
                          return_tensors='pt',
                          padding='max_length',
                          truncation=True,
                          max_length=2)['input_ids'][0][0].item()
    model.eval()

    result = np.array([])
    # <
    l = len(dataloader)
    # <
    for batch_id, batch in enumerate(dataloader):
        # <
#         print(batch)
        print(f'{batch_id} {datetime.datetime.now()}')
        # <
        tokens = model.generate(**batch)
#         print(tokens)
        tokens = [1 if pos_label in token else 0 for token in tokens]
#         print(tokens)
        result = np.hstack([result, tokens])
    
    return result

if TESTING_FROM_FILES:
    tokenizer_from_disk = T5Tokenizer.from_pretrained(SAVED_MODEL_PATH, use_fast=False)
    model_from_disk = T5ForConditionalGeneration.from_pretrained(SAVED_MODEL_PATH)
    model_from_disk.to(device)
    result = test(model_from_disk, tokenizer_from_disk, test_dataloader)
else:
    result = test(model, tokenizer, test_dataloader)

f1 = f1_score(result, test_df['l'])

print(f'T5 testing is done, F1-score: {f1:.3f}')

len
len
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
0 2023-08-27 16:35:39.268082


C:\Python310\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
1 2023-08-27 16:35:39.645482
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__getitem__
_tokenize
__g

## Saving

In [23]:

if os.path.isdir(SAVED_MODEL_PATH):
    for file_name in os.listdir(SAVED_MODEL_PATH):
        file_path = os.path.join(SAVED_MODEL_PATH, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(e)

tokenizer.save_pretrained(SAVED_MODEL_PATH)
tokenizer.save_vocabulary(SAVED_MODEL_PATH)
model.save_pretrained(SAVED_MODEL_PATH)

print('Saving is done.')

Saving is done.


## End